<a href="https://colab.research.google.com/github/vinayak2019/Polymer/blob/main/xyz2cif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ### **Install ASE**
#@markdown ---
#@markdown  Run this cell first.
!pip install ase

In [ ]:
from ase.io import read, write
import numpy as np


def build_periodic_polymer(
    xyz_file: str,
    output_file: str = "polymer.cif",
    dummy_symbol: str = "X",
    repeat: int = 1,
):
    """
    Build a periodic polymer unit cell from a monomer XYZ file.

    Parameters
    ----------
    xyz_file : str
        Path to the XYZ file of the monomer. The monomer must include exactly two dummy atoms
        labeled with `dummy_symbol` marking the polymer ends.
    output_file : str
        Path to write the periodic structure (CIF format recommended).
    dummy_symbol : str
        Chemical symbol used for dummy atoms (default: "X").
    repeat : int
        Number of monomers per cell along the polymer axis (default: 1).

    Output
    ------
    Writes a CIF file in which the first lattice vector aligns with the polymer axis
    and the other two span the true molecular extents perpendicular to it.
    """
    # Read monomer and identify dummy atoms
    mon = read(xyz_file, format="xyz")
    dummy_idxs = [i for i, a in enumerate(mon) if a.symbol == dummy_symbol]
    if len(dummy_idxs) != 2:
        raise ValueError(f"Expected exactly two dummy atoms ('{dummy_symbol}'), found {len(dummy_idxs)}")

    # Compute polymer axis from dummy positions
    d1, d2 = (mon.positions[i] for i in dummy_idxs)
    trans_vec = d2 - d1
    idx = np.argmax(np.abs(trans_vec))
    if trans_vec[idx] > 0:
        trans_vec[idx] -= 2.4
    else:
        trans_vec[idx] += 2.4
    length1 = np.linalg.norm(trans_vec)
    axis_dir = trans_vec / length1

    # Remove dummy atoms and recenter so that d1 is origin
    keep = [i for i in range(len(mon)) if i not in dummy_idxs]
    poly = mon[keep]
    poly.positions -= d1

    # Choose perpendicular directions
    tmp = np.array([1.0, 0.0, 0.0]) if abs(axis_dir[0]) < 0.9 else np.array([0.0, 1.0, 0.0])
    a2_dir = np.cross(axis_dir, tmp)
    a2_dir /= np.linalg.norm(a2_dir)
    a3_dir = np.cross(axis_dir, a2_dir)
    a3_dir /= np.linalg.norm(a3_dir)

    # Project atomic positions onto perpendicular axes
    coords = poly.get_positions()
    proj2 = coords.dot(a2_dir)
    proj3 = coords.dot(a3_dir)
    length2 = proj2.max() - proj2.min()
    length3 = proj3.max() - proj3.min()

    # Define lattice vectors
    a1 = axis_dir * length1 * repeat
    a2 = a2_dir * length2
    a3 = a3_dir * length3

    if length2 > length3:
        lattice = np.vstack([a2, a3, a1])
    else:
        lattice = np.vstack([a3, a2, a1])
    # Apply periodic cell
    poly.set_cell(lattice, scale_atoms=False)
    poly.set_pbc([True, True, True])

    # Optionally replicate along polymer axis
    # if repeat > 1:
    #     poly = poly.repeat((repeat, 1, 1))

    # Write output CIF
    write(output_file, poly)
    print(f"Written periodic polymer to {output_file}")




@markdown ### **Configuration**
#@markdown ---

molecule_xyz_filename = "mol.xyz" #@param {type:"string"}
output_cif_filename = "crystal.cif" #@param {type:"string"}
dummy_symbol = "X" #@param {type:"string"}


build_periodic_polymer(
    molecule_xyz_filename,
    output_file=".cif",
    dummy_symbol="X",
    repeat=1,
)
